In [ ]:
import urllib.request, re, pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
import logging
logfile = Path('C:/Users/***/Archive/Logs/Logfile.txt') # set to variable later
logging.basicConfig(filename=str(logfile),
                    filemode='a',
                    format='%(asctime)s,%(msecs)03d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG, force=True)

In [ ]:
logging.info("Running WinSCP_downloader")

In [ ]:
# move to json later
ftp_folder_url_w = '***/WEEKLY/' 
ftp_folder_url_m = '***/MONTHLY/'
ftp_folder_url_p = '***/2025/'

In [ ]:
pattern1 = re.compile(r'1064\s*(\d*)\s*')
pattern2 = re.compile(r'2024_.*?.zip')
pattern3 = re.compile(r'JTI_.*?.zip')

In [ ]:
def list_ftp_files_urllib(url):
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf8')
        files = data.splitlines()
        return files

In [ ]:
files_w = list_ftp_files_urllib(ftp_folder_url_w)
files_m = list_ftp_files_urllib(ftp_folder_url_m)
files_p = list_ftp_files_urllib(ftp_folder_url_p)

In [ ]:
FTP_list_w = []
for f in files_w:
    FTP_list_w.append([pattern1.findall(f),pattern2.findall(f)])
    
#--------------------------------------------------
FTP_list_m = []
for f in files_m:
    FTP_list_m.append([pattern1.findall(f),pattern2.findall(f)])

#--------------------------------------------------
FTP_list_p = []
for f in files_p:
    FTP_list_p.append([pattern1.findall(f),pattern3.findall(f)])

logging.info("WinSCP_downloader - FTP files gathered")

In [ ]:

%run XL_Loader.ipynb


In [ ]:
#MOVE FTP_List_w into dataframe and remove all [ ] things for comformable comparison

FTP_w = pd.DataFrame(FTP_list_w,columns=['size','name'])
FTP_w = FTP_w.astype(str)
#-------------------------------------
FTP_m = pd.DataFrame(FTP_list_m,columns=['size','name'])
FTP_m = FTP_m.astype(str)
#------------------------------------
FTP_p = pd.DataFrame(FTP_list_p,columns=['size','name'])
FTP_p = FTP_p.astype(str)
logging.info("WinSCP_downloader - FTP files loaded into df")

In [ ]:
for FTP in (FTP_w,FTP_m,FTP_p):
    if 'WEEKLY' in str(FTP['name']):
        FTP_w = FTP
        FTP_w['size'] = FTP_w['size'].str.replace("\['",'', regex=True)
        FTP_w['size'] = FTP_w['size'].str.replace("\']",'', regex=True)
        FTP_w['name'] = FTP_w['name'].str.replace("\['",'', regex=True)
        FTP_w['name'] = FTP_w['name'].str.replace("\']",'', regex=True)
        FTP_w = FTP_w.drop([0])
        FTP_w = FTP_w.reset_index()
        FTP_w = FTP_w.drop(columns=['index'])
        FTP_w = FTP_w[FTP_w['size'] !='5']
    
    elif 'MONTHLY' in str(FTP['name']):
        FTP_m = FTP
        FTP_m['size'] = FTP_m['size'].str.replace("\['",'', regex=True)
        FTP_m['size'] = FTP_m['size'].str.replace("\']",'', regex=True)
        FTP_m['name'] = FTP_m['name'].str.replace("\['",'', regex=True)
        FTP_m['name'] = FTP_m['name'].str.replace("\']",'', regex=True)
        FTP_m = FTP_m.drop([0])
        FTP_m = FTP_m.reset_index()
        FTP_m = FTP_m.drop(columns=['index'])
        FTP_m = FTP_m[FTP_m['size'] !='7']
        
    elif 'price' in str(FTP['name']):
        FTP_p = FTP
        FTP_p['size'] = FTP_p['size'].str.replace("\['",'', regex=True)
        FTP_p['size'] = FTP_p['size'].str.replace("\']",'', regex=True)
        FTP_p['name'] = FTP_p['name'].str.replace("\['",'', regex=True)
        FTP_p['name'] = FTP_p['name'].str.replace("\']",'', regex=True)
        FTP_p = FTP_p.drop([0])
        FTP_p = FTP_p.reset_index()
        FTP_p = FTP_p.drop(columns=['index'])
        FTP_p = FTP_p[FTP_p['size'] !='7']
        
logging.info("WinSCP_downloader - FTP files reformatted")

In [ ]:
excel_m = excel_m.astype(str)
excel_w = excel_w.astype(str)
excel_p = excel_p.astype(str)
excel_w = excel_w[excel_w['size'] !='5']
excel_m = excel_m[excel_m['size'] !='7']


In [ ]:
#WEEKLY LOAD AND CHECK
merge_w = pd.concat([FTP_w, excel_w], ignore_index = True)
merge_w = merge_w.drop_duplicates(keep = False) # we have unique name and size of a new file here
merge_p = pd.concat([FTP_p, excel_p], ignore_index = True)
merge_p = merge_p.drop_duplicates(keep = False) # we have unique name and size of a new file here
if merge_w.empty:
    print('No new weekly files on FTP')
    logging.info("WinSCP_downloader - No new weekly files on FTP")
    status_w = False # used for scheduler for him to know if load was successfull
else:
    print(f'New weekly files found on FTP - {merge_w} and {merge_p}, \n do you want to start download? y/n?')
    answer = input()
    if answer == 'y':
        try:
            download_f = merge_w['name'].iloc[0] #this is what we download!
            download_fp = merge_p['name'].iloc[0] 
            urllib.request.urlretrieve(f'***/WEEKLY/{download_f}',f'C:/Users/***/{download_f}')
            urllib.request.urlretrieve(f'***/2025/{download_fp}',f'C:/Users/***/{download_fp}')
            status_w = True # used for scheduler for him to know if load was successfull
            logging.debug("WinSCP_downloader - Weekly load successfull")
        except:
            print('Error during download')
            logging.debug("WinSCP_downloader - Error during download - weekly")
            download_f = None
    elif answer == 'n':
        try:
            sys.exit()
        except:
            print('Exiting')
            logging.debug("WinSCP_downloader - reply - N(weekly), exit program - no download occured")
    else:
        print('Please provide correct answer, y/n')


In [ ]:
#MONTHLY LOAD AND CHECK
merge_m = pd.concat([FTP_m, excel_m], ignore_index = True)
merge_m = merge_m.drop_duplicates(keep = False) # we have unique name and size of a new file here
if merge_m.empty:
    print('No new monthly files on FTP')
    logging.info("WinSCP_downloader - No new monthly files on FTP")
    status_m = False # used for scheduler for him to know if load was successfull
else: 
    print(f'New monthly file found on FTP - {merge_m},\n do you want to start download? y/n?')
    while True:
        answer = input()
        if answer == 'y':
            try:
                download_f = merge_m['name'].iloc[0] #this is what we download!
                urllib.request.urlretrieve(f'***/MONTHLY/{download_f}',f'C:/Users/***/{download_f}')
                status_m = True # used for scheduler for him to know if load was successfull
            except:
                print('Error during download')
                logging.debug("WinSCP_downloader - Error during download - monthly")
                download_f = None
        elif answer == 'n':
            try:
                sys.exit()
            except:
                print('Exiting')
                logging.debug("WinSCP_downloader - reply - N (monthly), exit program - no download occured")
        else:
            print('Please provide correct answer, y/n')


In [ ]:
#move to json later
 
FTP_w.to_excel('C:/Users/***/FTP/CMR_W.xlsx')
FTP_m.to_excel('C:/Users/***/FTP/CMR_M.xlsx')
FTP_p.to_excel('C:/Users/***/FTP/jti_price_surowe.xlsx')
logging.debug("WinSCP_downloader - updating excel FTP reference list")

In [ ]:
if status_w == True or status_m == True:
    %run Unzip_files.ipynb
else:
    None

In [ ]:
logging.info("WinSCP_downloader finished working successfully")